# NDVI -  Como NO debería ser
Implementación de generación del índice de vegetación (ndvi) utilizando las consultas a bajo nivel que ofrece el cubo. 
## Parámetros

In [1]:
%matplotlib inline
longs=(-75, -74)
lats=(5, 6)
normalized=True
time_range=("2000-01-01","2000-12-31")
bands=["blue","green","red","nir", "swir1","swir2"]
pdct='ls7_ledaps_utm'

## CONSULTA

In [2]:
import datacube
import numpy as np
dc = datacube.Datacube(app='dc-prueba2')
minValid=1;
nbar = dc.load(product=pdct, longitude=longs, latitude=lats, time=time_range)

In [3]:
## Medianas

In [4]:
nodata=-9999
medians={}
for band in bands:
    
    datos=np.where(nbar.data_vars[band]!=-9999,nbar.data_vars[band], np.nan)
    allNan=~np.isnan(datos)
    if normalized:
        m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
        st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
        datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
    medians[band]=np.nanmedian(datos,0)
    medians[band][np.sum(allNan,0)<minValid]=np.nan
del datos    

/home/developer/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:740: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


## NDVI

In [5]:
period_red = medians["red"]
period_nir = medians["nir"]
del medians
mask_nan=np.logical_or(np.isnan(period_red), np.isnan(period_nir))
period_nvdi = np.true_divide( np.subtract(period_nir,period_red) , np.add(period_nir,period_red) )
period_nvdi[mask_nan]=np.nan

## Guardar a archivo NetCDF

In [6]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
nco=netcdf_writer.create_netcdf("ndvi2000_1.nc")
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, nbar.coords[x].values, nbar.coords[x].units)
    else:
        netcdf_writer.create_coordinate(nco, x, np.array(["t1-t2"]), "period")
netcdf_writer.create_grid_mapping_variable(nco, nbar.crs)
period_nvdi[np.isnan(period_nvdi)]=nodata
var= netcdf_writer.create_variable(nco, 'ndvi', Variable(np.dtype(np.float32), None, ('time','latitude', 'longitude'), None) ,set_crs=True)
var[:] = netcdf_writer.netcdfy_data(period_nvdi)
nco.close()